In [11]:
from src.data_preparation.data_preparation import read_eeg_file
from scipy import signal
from scipy import linalg
import pyriemann.utils.mean as rie_mean
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score
import numpy as np

Define the constants with parameters to apply the algorithm

In [12]:
FS = 250
TIME_LENGTH = 2 * FS
TIME_WINDOW = 2 * FS
EPOCH_SIZE = None
DATA_FOLDER = "data/csp/bci-iv-dataset-ii-b"
CSP_COMPONENTS = 2

Define the objects to store the evaluation data 

In [13]:
subjects = range(1, 10)
accuracies = {
    "SVM": np.zeros(len(subjects)),
    "LDA": np.zeros(len(subjects))
}

Define the bandpass filter to be used in the pre-processing step

In [14]:
sos = signal.butter(5, [8, 30], analog=False, btype="band", output="sos", fs=FS)

Load the EEG data

In [15]:
data = dict()
data["train"] = []
data["test"] = []
for subject in subjects:
    left_data_file = f"{DATA_FOLDER}/left-hand-train-subject-{subject}.csv"
    right_data_file = f"{DATA_FOLDER}/right-hand-train-subject-{subject}.csv"
    data["train"].append(read_eeg_file(left_data_file, right_data_file, TIME_LENGTH, TIME_WINDOW, EPOCH_SIZE))
    
    left_data_file = f"{DATA_FOLDER}/left-hand-test-subject-{subject}.csv"
    right_data_file = f"{DATA_FOLDER}/right-hand-test-subject-{subject}.csv"
    data["test"].append(read_eeg_file(left_data_file, right_data_file, TIME_LENGTH, TIME_WINDOW, EPOCH_SIZE))

Define the function to generate the common spatial filter's based on the test data 

In [17]:
def compute_spatial_filters(train_data):
    cov_shape = (train_data.n_channels, train_data.n_channels)
    
    # Estimate the covariance matrix of every trial
    cov = np.zeros((train_data.n_left_trials, *cov_shape))
    for n_trial in range(train_data.n_left_trials):
        trial_filtered = signal.sosfilt(sos, train_data.left_data[n_trial], axis=0)
        cov[n_trial,:,:] = np.cov(np.transpose(trial_filtered))

    # calculate average of covariance matrix
    cov_1 = rie_mean.mean_covariance(cov, metric="riemann")
    
    # Estimate the covariance matrix of every trial
    cov = np.zeros((train_data.n_right_trials, *cov_shape))
    for n_trial in range(train_data.n_right_trials):
        trial_filtered = signal.sosfilt(sos, train_data.right_data[n_trial], axis=0)
        cov[n_trial,:,:] = np.cov(np.transpose(trial_filtered))
        
    # calculate average of covariance matrix
    cov_2 = rie_mean.mean_covariance(cov, metric="riemann")
    
    # Solve the generalized eigenvalue problem
    n_pairs = CSP_COMPONENTS//2
    w, vr = linalg.eig(cov_1, cov_2, right=True)
    w = np.abs(w)
    sorted_indexes = np.argsort(w)
    chosen_indexes = np.zeros(2*n_pairs).astype(int)
    chosen_indexes[0:n_pairs] = sorted_indexes[0:n_pairs]
    chosen_indexes[n_pairs:2*n_pairs] = sorted_indexes[-n_pairs:]

    return vr[:, chosen_indexes]

Define the function to apply the spatial filter and extract the features 

In [18]:
def extract_features(X, W):
    trials = len(X)
    F = np.zeros((trials, CSP_COMPONENTS))
    for n_trial in range(trials):
        trial = X[n_trial]
        Z = np.dot(np.transpose(W), np.transpose(trial))
        Z = signal.sosfilt(sos, Z, axis=1)
        F[n_trial] = np.log(np.divide(np.var(Z, axis=1), np.sum(np.var(Z, axis=1))))
        
    return F

Iterate on the subjects, training the model using the 3 training sessions 
and evaluating the model using the 2 evaluation sessions

In [19]:
for subject in subjects:
    print(f"Subject {subject} ...")
    subject_index = subject - 1
    train_data = data["train"][subject_index]
    test_data = data["test"][subject_index]

    W = compute_spatial_filters(train_data)
    
    features = dict({
        "train": extract_features(train_data.X, W),
        "test": extract_features(test_data.X, W)
    })
    
    # SVM classifier
    svm = SVC(C=.8, kernel="rbf", gamma="scale")
    svm.fit(features["train"], train_data.labels)
    svm_predictions = svm.predict(features["test"])
    accuracies["SVM"][subject_index] = accuracy_score(test_data.labels, svm_predictions)
    
    # LDA classifier
    lda = LinearDiscriminantAnalysis()
    lda.fit(features["train"], train_data.labels)
    lda_predictions = lda.predict(features["test"])
    accuracies["LDA"][subject_index] = accuracy_score(test_data.labels, lda_predictions)
    
print("\nAccuracy")
for classifier in accuracies:
    print(classifier)
    for subject, cv_accuracies in enumerate(accuracies[classifier]):
        acc_mean = np.mean(cv_accuracies)
        print(f"\tSubject {subject+1} average accuracy: {acc_mean:.4f}")
    average_acc_mean = np.mean(accuracies[classifier])
    print(f"\tAverage accuracy: {average_acc_mean:.4f}")
    

Subject 1 ...
Subject 2 ...
Subject 3 ...
Subject 4 ...
Subject 5 ...
Subject 6 ...
Subject 7 ...
Subject 8 ...
Subject 9 ...

Accuracy
SVM
	Subject 1 average accuracy: 0.6316
	Subject 2 average accuracy: 0.5918
	Subject 3 average accuracy: 0.5130
	Subject 4 average accuracy: 0.9642
	Subject 5 average accuracy: 0.6960
	Subject 6 average accuracy: 0.8048
	Subject 7 average accuracy: 0.7974
	Subject 8 average accuracy: 0.9261
	Subject 9 average accuracy: 0.8245
	Average accuracy: 0.7499
LDA
	Subject 1 average accuracy: 0.6316
	Subject 2 average accuracy: 0.6000
	Subject 3 average accuracy: 0.5522
	Subject 4 average accuracy: 0.9446
	Subject 5 average accuracy: 0.6813
	Subject 6 average accuracy: 0.7849
	Subject 7 average accuracy: 0.7845
	Subject 8 average accuracy: 0.9348
	Subject 9 average accuracy: 0.8000
	Average accuracy: 0.7460
